<a href="https://colab.research.google.com/github/Shreyes45/BIS-lab/blob/main/BIS-lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

def total_distance(routes, dist_matrix):
    dist = 0
    for route in routes:
        for i in range(len(route)-1):
            dist += dist_matrix[route[i], route[i+1]]
    return dist

def generate_random_routes(num_customers, num_vehicles):
    customers = np.arange(1, num_customers + 1)
    np.random.shuffle(customers)
    splits = np.array_split(customers, num_vehicles)
    return [[0] + list(s) + [0] for s in splits if len(s) > 0]

def crossover(r1, r2):
    flat1 = np.concatenate([x[1:-1] for x in r1 if len(x)>2]) if r1 else []
    flat2 = np.concatenate([x[1:-1] for x in r2 if len(x)>2]) if r2 else []
    if len(flat1)==0 or len(flat2)==0:
        return r1 if len(flat1)>0 else r2
    point = np.random.randint(1, len(flat1))
    child_seq = list(flat1[:point]) + [x for x in flat2 if x not in flat1[:point]]
    splits = np.array_split(child_seq, 2)
    return [[0]+list(s)+[0] for s in splits if len(s)>0]

def mutate(routes, rate=0.2):
    for route in routes:
        if np.random.rand() < rate and len(route) > 3:
            i, j = np.random.choice(range(1, len(route)-1), 2, replace=False)
            route[i], route[j] = route[j], route[i]
    return routes

def local_optimize(routes, dist_matrix):
    for route in routes:
        improved = True
        while improved:
            improved = False
            for i in range(1, len(route)-2):
                for j in range(i+1, len(route)-1):
                    a, b, c, d = route[i-1], route[i], route[j], route[j+1]
                    before = dist_matrix[a,b]+dist_matrix[c,d]
                    after = dist_matrix[a,c]+dist_matrix[b,d]
                    if after < before:
                        route[i:j+1] = route[i:j+1][::-1]
                        improved = True
    return routes

def create_distance_matrix(num_customers):
    coords = np.random.rand(num_customers + 1, 2) * 20
    return np.sqrt(((coords[:, None, :] - coords[None, :, :]) ** 2).sum(-1))

def parallel_cellular_vrp(num_customers=10, num_vehicles=2, grid_size=(3,3), iterations=50):
    dist_matrix = create_distance_matrix(num_customers)
    grid = np.empty(grid_size, dtype=object)
    fit = np.zeros(grid_size)

    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            routes = generate_random_routes(num_customers, num_vehicles)
            grid[i,j] = routes
            fit[i,j] = 1 / (1 + total_distance(routes, dist_matrix))

    best = grid[np.unravel_index(np.argmax(fit), fit.shape)]

    for _ in range(iterations):
        new_grid = np.empty_like(grid)
        for i in range(grid_size[0]):
            for j in range(grid_size[1]):
                neighbors = []
                for di in [-1,0,1]:
                    for dj in [-1,0,1]:
                        if di==0 and dj==0: continue
                        ni, nj = (i+di)%grid_size[0], (j+dj)%grid_size[1]
                        neighbors.append(grid[ni,nj])
                best_neighbor = max(neighbors, key=lambda r: 1/(1+total_distance(r, dist_matrix)))
                child = crossover(grid[i,j], best_neighbor)
                child = mutate(child)
                child = local_optimize(child, dist_matrix)
                new_fit = 1 / (1 + total_distance(child, dist_matrix))
                if new_fit > fit[i,j]:
                    new_grid[i,j] = child
                    fit[i,j] = new_fit
                else:
                    new_grid[i,j] = grid[i,j]
        grid = new_grid
        current_best = grid[np.unravel_index(np.argmax(fit), fit.shape)]
        if (1/(1+total_distance(current_best, dist_matrix))) > (1/(1+total_distance(best, dist_matrix))):
            best = current_best

    return best, total_distance(best, dist_matrix)

best_routes, best_cost = parallel_cellular_vrp()
print("Best Routes:", best_routes)
print("Total Distance:", round(best_cost, 2))


Best Routes: [[0, np.int64(6), np.int64(3), np.int64(7), np.int64(1), np.int64(4), 0], [0, np.int64(8), np.int64(5), np.int64(2), np.int64(9), np.int64(10), 0]]
Total Distance: 86.92
